In [44]:
import os
import shutil
import stat

In [45]:
def get_file_list(path: str) -> list:
    """ Returns a list of files in a directory 
    
    Arrgs:
        path (str): path to the directory

    Returns:
        file_list (list): list of files in the directory
    """
    file_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

In [46]:
def get_unique_file_types(file_list: list) -> list:
    """ Identifies the type of file and returns a dictionary with the file type and the list of files

    Args:
        file_list (list): list of files

    Returns:
        unique_file_types (list): List of file types
    """
    unique_file_types = set()
    for file in file_list:
        file_type = file.split('.')[-1]
        unique_file_types.add(file_type)
    
    return unique_file_types

In [47]:
def remove_readonly(func, path, _):
    """Clears the read-only attribute and retries deletion."""
    os.chmod(path, stat.S_IWRITE)
    func(path)

def create_folders(unique_file_types: list) -> None:
    if os.path.exists("images"):
        shutil.rmtree("images", onerror=remove_readonly)
    
    if os.path.exists("documents"):
        shutil.rmtree("documents", onerror=remove_readonly)
    
    if os.path.exists("codes"):
        shutil.rmtree("codes", onerror=remove_readonly)

    for file_type in unique_file_types:
        if file_type in ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'svg', 'ico']:
            os.makedirs("images", exist_ok=True)
        
        elif file_type in ['txt', 'doc', 'docx', 'pdf', 'ppt', 'pptx', 'xls', 'xlsx', 'csv']:
            os.makedirs("documents", exist_ok=True)

        elif file_type in ['py', 'ipynb']:
            os.makedirs("codes", exist_ok=True)

In [48]:
def move_files_to_folder(source_folder):
    for filename in os.listdir(source_folder):
        file_extension = filename.split('.')[-1]
        if file_extension in  ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'svg', 'ico']:
            print(source_folder, filename)
            shutil.move(os.path.join(source_folder, filename), os.path.join("images"))
        elif file_extension in  ['txt', 'doc', 'docx', 'pdf', 'ppt', 'pptx', 'xls', 'xlsx', 'csv']:
            shutil.move(os.path.join(source_folder, filename), os.path.join("documents"))
        elif file_extension in ['py', 'ipynb']:
            shutil.move(os.path.join(source_folder, filename), os.path.join("codes"))

In [49]:
files_list = get_file_list("un_organized")
unique_file_types = get_unique_file_types(files_list)
create_folders(unique_file_types)
move_files_to_folder("un_organized")

un_organized image_1.jpg
un_organized image_2.jpg
un_organized image_3.png
un_organized image_4.svg


In [52]:
import os
import shutil
import stat
from typing import List, Set, Dict
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define file type mappings
FILE_TYPE_MAPPINGS = {
    'images': {'jpg', 'jpeg', 'png', 'gif', 'bmp', 'svg', 'ico'},
    'documents': {'txt', 'doc', 'docx', 'pdf', 'ppt', 'pptx', 'xls', 'xlsx', 'csv'},
    'codes': {'py', 'ipynb'}
}

def get_file_list(path: str) -> List[str]:
    """
    Returns a list of files in a directory and its subdirectories.
    
    Args:
        path (str): Path to the directory to scan.

    Returns:
        List[str]: List of absolute paths to all files in the directory.

    Raises:
        FileNotFoundError: If the specified path doesn't exist.
        PermissionError: If there are insufficient permissions to access the directory.
    """
    try:
        path = os.path.abspath(path)
        if not os.path.exists(path):
            raise FileNotFoundError(f"Directory not found: {path}")
        
        file_list = []
        for root, _, files in os.walk(path):
            for file in files:
                file_list.append(os.path.join(root, file))
        
        logging.info(f"Found {len(file_list)} files in {path}")
        return file_list
    
    except PermissionError as e:
        logging.error(f"Permission denied accessing {path}: {str(e)}")
        raise
    except Exception as e:
        logging.error(f"Error scanning directory {path}: {str(e)}")
        raise

def get_unique_file_types(file_list: List[str]) -> Set[str]:
    """
    Extracts unique file extensions from a list of file paths.

    Args:
        file_list (List[str]): List of file paths.

    Returns:
        Set[str]: Set of unique file extensions (lowercase).
    """
    unique_file_types = set()
    for file in file_list:
        # Handle files without extensions
        if '.' in file:
            file_type = file.split('.')[-1].lower()
            unique_file_types.add(file_type)
    
    logging.info(f"Found {len(unique_file_types)} unique file types: {unique_file_types}")
    return unique_file_types

def remove_readonly(func, path, _):
    """
    Clears the read-only attribute and retries the file operation.
    
    Args:
        func: The original function that failed (typically os.remove or os.rmdir)
        path (str): Path to the file/directory
        _: ExcInfo triple (not used)
    """
    try:
        os.chmod(path, stat.S_IWRITE)
        func(path)
    except Exception as e:
        logging.error(f"Failed to remove read-only file/directory {path}: {str(e)}")
        raise

def create_folders(unique_file_types: Set[str], base_path: str = ".") -> Dict[str, str]:
    """
    Creates organized folders based on file types and returns their paths.
    
    Args:
        unique_file_types (Set[str]): Set of file extensions to organize
        base_path (str): Base directory for creating organized folders

    Returns:
        Dict[str, str]: Mapping of folder categories to their paths

    Raises:
        PermissionError: If there are insufficient permissions to create directories
    """
    folder_paths = {}
    
    try:
        # Create base path if it doesn't exist
        os.makedirs(base_path, exist_ok=True)
        
        # Determine which folders need to be created based on file types
        needed_folders = set()
        for folder_name, extensions in FILE_TYPE_MAPPINGS.items():
            if any(ext in unique_file_types for ext in extensions):
                needed_folders.add(folder_name)
        
        # Remove and recreate needed folders
        for folder in needed_folders:
            folder_path = os.path.join(base_path, folder)
            if os.path.exists(folder_path):
                shutil.rmtree(folder_path, onerror=remove_readonly)
            os.makedirs(folder_path)
            folder_paths[folder] = folder_path
            logging.info(f"Created folder: {folder_path}")
        
        return folder_paths
    
    except PermissionError as e:
        logging.error(f"Permission denied creating folders in {base_path}: {str(e)}")
        raise
    except Exception as e:
        logging.error(f"Error creating folders: {str(e)}")
        raise

def move_files_to_folder(source_folder: str, folder_paths: Dict[str, str]) -> None:
    """
    Moves files from source folder to their respective organized folders.
    
    Args:
        source_folder (str): Path to the source folder containing files to organize
        folder_paths (Dict[str, str]): Mapping of folder categories to their paths

    Raises:
        FileNotFoundError: If source folder doesn't exist
        PermissionError: If there are insufficient permissions
    """
    try:
        if not os.path.exists(source_folder):
            raise FileNotFoundError(f"Source folder not found: {source_folder}")
        
        for filename in os.listdir(source_folder):
            if '.' not in filename:
                continue
                
            file_extension = filename.split('.')[-1].lower()
            source_path = os.path.join(source_folder, filename)
            
            # Determine destination folder
            dest_folder = None
            for folder_name, extensions in FILE_TYPE_MAPPINGS.items():
                if file_extension in extensions:
                    dest_folder = folder_paths.get(folder_name)
                    break
            
            if dest_folder:
                dest_path = os.path.join(dest_folder, filename)
                shutil.move(source_path, dest_path)
                logging.info(f"Moved {filename} to {dest_folder}")
    
    except (FileNotFoundError, PermissionError) as e:
        logging.error(f"Error accessing files: {str(e)}")
        raise
    except Exception as e:
        logging.error(f"Error moving files: {str(e)}")
        raise

def organize_files(source_path: str, base_path: str = ".") -> None:
    """
    Main function to organize files from source directory into categorized folders.
    
    Args:
        source_path (str): Path to the source directory containing files to organize
        base_path (str): Base directory for creating organized folders
    """
    try:
        files_list = get_file_list(source_path)
        unique_file_types = get_unique_file_types(files_list)
        folder_paths = create_folders(unique_file_types, base_path)
        move_files_to_folder(source_path, folder_paths)
        logging.info("File organization completed successfully")
    
    except Exception as e:
        logging.error(f"File organization failed: {str(e)}")
        raise

# Example usage
if __name__ == "__main__":
    organize_files("un_organized")

2025-02-13 18:45:40,624 - INFO - Found 12 files in c:\Users\achakravarti\OneDrive - ALTEN Group\Personal and Otherstuff\Learnings\EPAi5\EPAiV5-CapStone\prototype_2\un_organized
2025-02-13 18:45:40,625 - INFO - Found 8 unique file types: {'jpg', 'xlsx', 'docx', 'txt', 'svg', 'png', 'pptx', 'py'}
2025-02-13 18:45:40,627 - INFO - Created folder: .\codes
2025-02-13 18:45:40,628 - INFO - Created folder: .\images
2025-02-13 18:45:40,629 - INFO - Created folder: .\documents
2025-02-13 18:45:40,631 - INFO - Moved excel_1.xlsx to .\documents
2025-02-13 18:45:40,633 - INFO - Moved image_1.jpg to .\images
2025-02-13 18:45:40,635 - INFO - Moved image_2.jpg to .\images
2025-02-13 18:45:40,637 - INFO - Moved image_3.png to .\images
2025-02-13 18:45:40,638 - INFO - Moved image_4.svg to .\images
2025-02-13 18:45:40,639 - INFO - Moved power_point_1.pptx to .\documents
2025-02-13 18:45:40,641 - INFO - Moved python_1.py to .\codes
2025-02-13 18:45:40,642 - INFO - Moved python_2.py to .\codes
2025-02-13 1

In [53]:
import os
import requests

# Input and output folders
INPUT_FOLDER = "images"  # Change this to your source folder
OUTPUT_FOLDER = "compressed_images"

# Ensure output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Supported image formats
VALID_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"}

# reSmush.it API URL
API_URL = "https://api.resmush.it/ws.php"

def compress_image(image_path, output_path, quality=80):
    """Compress a single image using reSmush.it API."""
    with open(image_path, "rb") as img_file:
        files = {"files": img_file}
        params = {"qlty": quality}  # Set quality (1-100)
        response = requests.post(API_URL, files=files, params=params)

    if response.status_code == 200:
        data = response.json()
        if "dest" in data:
            # Download optimized image
            img_data = requests.get(data["dest"]).content
            with open(output_path, "wb") as out_file:
                out_file.write(img_data)
            print(f"Compressed: {image_path} -> {output_path}")
        else:
            print(f"Failed to compress {image_path}: {data}")
    else:
        print(f"Error {response.status_code} for {image_path}")

# Process all images in the folder
for filename in os.listdir(INPUT_FOLDER):
    file_path = os.path.join(INPUT_FOLDER, filename)
    ext = os.path.splitext(filename)[-1].lower()
    
    if os.path.isfile(file_path) and ext in VALID_EXTENSIONS:
        output_file = os.path.join(OUTPUT_FOLDER, filename)
        compress_image(file_path, output_file)

print("Image compression completed!")


Compressed: images\image_1.jpg -> compressed_images\image_1.jpg
Compressed: images\image_2.jpg -> compressed_images\image_2.jpg
Compressed: images\image_3.png -> compressed_images\image_3.png
Image compression completed!
